# Scrapping Data :

In [62]:
# import libraries
import pandas as pd
import requests, re
from bs4 import BeautifulSoup
from scrapfly import ScrapflyClient, ScrapeConfig
import time

In [27]:
recipes_content = []
id_ = 1

for i in range(1, 9) :
    url = f"https://www.deliciousmagazine.co.uk/cuisine/moroccan-recipes/page/{i}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # all_recipes = soup.find_all('div', {'class': 'recipe-list recipe-archive'})
    all_recipes = soup.find_all('div', {'class': 'grid-item recipe-card-wrapper'})

    for recipe in all_recipes :
        body_card = recipe.find('div', {'class':'card-body'})
        all_links = body_card.find_all('a')
        if len(all_links) <= 1:
            continue
        recipe_link = all_links[1].attrs['href']
        # print(recipe_link)
        #-------------------------
        response2 = requests.get(recipe_link)
        soup2 =  BeautifulSoup(response2.content, 'html.parser')
        recipe_content = soup2.find('main', {'class':'main-content content'})
        #-------------------------
        description_container = recipe_content.find('div', {'class':'recipe-detail-wrapper'})
        description = description_container.find('div', {'class':'recipe-description text-standard'}).text.strip().replace('\n', '')
        # print(description)
        #-------------------------
        ingredients_container = recipe_content.find('div', {'id':'ingredients'})
        ingredients_content = ingredients_container.find('ul')
        ingredients = [li.text for li in ingredients_content.find_all('li')]
        # print(ingredients)
        #-------------------------
        method_container = recipe_content.find('div', {'id':'method'})
        steps = None
        if not isinstance(method_container, type(None)):
            method_content = method_container.find('ol')
            steps = [li.text for li in method_content.find_all('li')]
        # print(steps)
        #-------------------------
        recipes_content.append({
            'id': id_, 
            'title': recipe.find('h4', {'class': 'card-title heading-std-text'}).text.strip(), 
            'description': description, 
            'ingredients': ingredients, 
            'steps': steps,
        })
        id_ += 1

In [28]:
data1 = pd.DataFrame(recipes_content)
data1.head()

,id,title,description,ingredients,steps
0,1,Duck and prune tagine,Duck legs are the ultimate choice for low-and-...,"[2 tbsp olive oil, 2 cinnamon sticks, 3 red o...",[Pour the oil into a heavy-based pan or casser...
1,2,Potato tagine with baked halloumi,This North African-inspired potato dish is by ...,"[2 tbsp vegetable oil, 2 onions, sliced, 6 gar...",[Heat the oven to 180°C fan/gas 6. Heat the oi...
2,3,Merguez sausages with crispy chickpeas and pre...,"Merguez sausages with crispy, roasted chickpea...","[2 x 400g tins chickpeas, drained, 3 tbsp oliv...",[Heat the oven to 180°C fan/gas 6. Put the chi...
3,4,Chicken tagine casserole,Enjoy all the rich flavours of a chicken tagin...,"[8 small free-range chicken thighs, 2 medium r...",[Heat the oven to 180ºC fan/gas 6. Slash the s...
4,5,Preserved lemons,With only two basic ingredients and some imagi...,[12 small unwaxed lemons (about 1kg; see tips)...,[Juice 4 of the lemons and set the juice aside...


In [29]:
data1.shape

(93, 5)

In [30]:
data1.drop(columns='id', axis=1, inplace=True)
data1.columns

Index(['title', 'description', 'ingredients', 'steps'], dtype='object')

In [31]:
data1.to_csv('Cleaned Data/Moroccan Data/scrapped_moroccan_recipes1.csv', index=False)

---

In [65]:
client = ScrapflyClient(key="scp-live-32ce28606fc94171b283a3459c6f32ff")

def get_html_content(url):
    result = client.scrape(ScrapeConfig(url=url, asp=True, render_js=True))
    if result.status_code != 200:
        print("Error:", result.error)
        return None
    return result.content

In [66]:
url2 = "https://www.thespruceeats.com/moroccan-main-dishes-4162467"
response2 = get_html_content(url2)

if response2:
    soup2 = BeautifulSoup(response2, 'html.parser')
    all_recipes2 = soup2.find('div', {'id': 'mntl-taxonomysc-article-list-group_1-0'})
    all_recipes2 = all_recipes2.find_all('a')
    print(len(all_recipes2))

    recipes_content2 = []
    id2 = 1

    for recipe in all_recipes2:
        recipe_link = recipe.attrs['href']
        
        response_2 = get_html_content(recipe_link)
        if response_2:
            soup_2 = BeautifulSoup(response_2, 'html.parser')
            recipe_content = soup_2.find('article', {'id': 'article--structured-project_1-0'})
            
            if recipe_content:
                title = recipe_content.find('h1', {'class': 'heading__title'}).text.strip()
                print(title)

                total_time_container = recipe_content.find('div', {'id': 'project-meta_1-0'})
                total_time_container = total_time_container.find('div', {'class': 'total-time project-meta__total-time'})
                total_time = total_time_container.find('span', {'class': 'meta-text__data'}).text.strip()

                description_container = recipe_content.find('div', {'id': 'article__header--project_1-0'})
                all_paragraphs = description_container.find_all('p')
                description = ''.join([p.get_text().strip().replace('\n', '') for p in all_paragraphs])

                ingredients_container = recipe_content.find('section', {'id': 'section--ingredients_1-0'})
                ingredients_content = ingredients_container.find('ul')
                ingredients = [li.text.replace('\n', '') for li in ingredients_content.find_all('li')]

                steps_container = recipe_content.find('section', {'id': 'section--instructions_1-0'})
                steps_dict = {}
                h2_elements = steps_container.find_all('h2')
                for h2 in h2_elements:
                    h2_text = h2.get_text(strip=True)
                    ol = h2.find_next('ol')
                    if ol:
                        steps = [li.get_text(strip=True) for li in ol.find_all('li')]
                        steps_dict[h2_text] = steps

                recipes_content2.append({
                    'id': id2,
                    'title': title,
                    'description': description,
                    'total_time': total_time,
                    'ingredients': ingredients,
                    'steps': steps_dict,
                })
                id2 += 1
        
        time.sleep(5)  # Add a delay to prevent rapid requests

58
Pressure Cooker Moroccan Lamb or Beef Tagine With Prunes
Moroccan Chicken Tagine with Olives and Preserved Lemons
Moroccan Chermoula Marinade
Moroccan Roasted Chicken With Preserved Lemons
Moroccan Chicken Rfissa - Trid Pastry With Chicken, Lentils, and Fenugreek
Seffa Medfouna Recipe (Vermicelli With Saffron Chicken)
Moroccan Chicken Tagine With Nigella Seeds (Black Seeds or Sanouj)
Moroccan Baked Fish Tagine With Potatoes, Carrots, Tomatoes and Peppers Recipe
Moroccan Tagine With Carrots, Potatoes, and Chickpeas Recipe
Moroccan Harcha Semolina Pan-Fried Flatbread
Moroccan Lamb or Beef Tagine with Apricots
Moroccan Fish Tagine (Mqualli)
Spicy Moroccan Rice with Tomatoes and Peppers
Moroccan Couscous With Meat and Seven Vegetables
Moroccan Vegetarian Carrot and Chickpea Tagine
Moroccan Grilled Lamb Chops With Garlic, Mint and Spices Recipe
Moroccan Saffron Chicken
Classic Moroccan Lamb or Beef Kebabs (Brochettes)
Moroccan Black-Eyed Peas (Cowpeas) - Ful Gnaoua
Moroccan Chicken and A

In [71]:
data2 = pd.DataFrame(recipes_content2)
data2.head()

,id,title,description,total_time,ingredients,steps
0,1,Pressure Cooker Moroccan Lamb or Beef Tagine W...,Beef or lamb with prunes is a classic sweet an...,85 mins,"[2 pounds tender lamb, or beef, cut into 3-inc...",{'Steps to Make It': ['Gather the ingredients....
1,2,Moroccan Chicken Tagine with Olives and Preser...,"A classic dish, this Moroccan chicken tagine g...",5 hrs 10 mins,"[1 to 2 preserved lemons, quartered and seeds ...",{'Steps to Make It': ['Gather the ingredients....
2,3,Moroccan Chermoula Marinade,Chermoula is a thick marinade for fish and the...,10 mins,"[1 large bunch cilantro, finely chopped (only ...",{'Steps to Make It': ['Gather the ingredients....
3,4,Moroccan Roasted Chicken With Preserved Lemons,Chicken with Preserved Lemon and Olives is a c...,3 hrs 20 mins,"[1 whole chicken, 1/2 teaspoon salt, 1/2 teasp...",{'Steps to Make It': ['Make a spice rub for th...
4,5,Moroccan Chicken Rfissa - Trid Pastry With Chi...,There may be nothing elegant about pouring hot...,3 hrs 30 mins,"[1 chicken, quartered, or left whole, 2 to 3 l...",{'Steps to Make It': ['To reduce their pungenc...


In [72]:
data2.shape

(56, 6)

In [73]:
data2.drop(columns='id', axis=1, inplace=True)
data2.columns

Index(['title', 'description', 'total_time', 'ingredients', 'steps'], dtype='object')

In [74]:
data2.to_csv('Cleaned Data/Moroccan Data/scrapped_moroccan_recipes2.csv', index=False)

---

In [87]:
recipes_content3 = []
id3 = 1

for i in range(5) :
    offset = i * 24
    # offset = 0
    url3 = f"https://www.allrecipes.com/search?moroccan=moroccan&offset={offset}&q=moroccan"
    response3 = get_html_content(url3)
    soup3 = BeautifulSoup(response3, 'html.parser')

    recipes_container = soup3.find('div', {'id': 'mntl-search-results__list_1-0'})
    all_recipes_links = recipes_container.find_all('a')

    for recipe in all_recipes_links :
        recipe_link = recipe.attrs['href']
        # print(recipe_link)
        response_3 = get_html_content(recipe_link)
        if response_3:
            soup_3 = BeautifulSoup(response_3, 'html.parser')
            recipe_content = soup_3.find('article', {'id': 'allrecipes-article_1-0'})
            
            if recipe_content:
                title_describtion_container = recipe_content.find('div', {'id': 'article-header--recipe_1-0'})
                # -------------------------
                title = title_describtion_container.find('h1', {'class': 'article-heading type--lion'}).text.strip()
                print(title)
                # -------------------------
                description = title_describtion_container.find('p', {'class': 'article-subheading type--dog'}).text.strip()
                # print(description)
                # -------------------------
                total_time_container = recipe_content.find('div', {'id': 'mm-recipes-details_1-0'})
                total_time_container = total_time_container.find('div', {'class': 'mm-recipes-details__item'})
                total_time = total_time_container.find('div', {'class': 'mm-recipes-details__value'}).text.strip()
                # print(total_time)
                # -------------------------
                ingredients_container = recipe_content.find('div', {'id': 'mm-recipes-structured-ingredients_1-0'})
                ingredients_content = ingredients_container.find('ul')
                ingredients = [li.text.replace('\n', '') for li in ingredients_content.find_all('li')]
                # print(ingredients)
                # -------------------------
                steps_container = recipe_content.find('div', {'id': 'mm-recipes-steps__content_1-0'})
                steps_content = steps_container.find('ol')
                steps = [li.text.replace('\n', '') for li in steps_content.find_all('li')]
                # print(steps)
                # -------------------------
                recipes_content3.append({
                    'id': id3,
                    'title': title,
                    'description': description,
                    'total_time': total_time,
                    'ingredients': ingredients,
                    'steps': steps,
                })
                id3 += 1
                # print('//'*50)
        # time.sleep(2)

Moroccan Couscous
Make-Ahead Vegetarian Moroccan Stew
Easy Moroccan Chicken Tagine
Spanish Moroccan Fish
Moroccan Chicken
Moroccan Lentil Soup with Veggies
Moroccan-Style Stuffed Acorn Squash
Moroccan Chickpea Stew
Sweet and Nutty Moroccan Couscous
Moroccan Chicken and Whole Grain Couscous
Moroccan Lentil Soup
Cara's Moroccan Stew
Moroccan Lamb with Shiraz Honey Sauce
Slow Cooker Moroccan Chicken
Moroccan Harira (Bean Soup)
Make-Ahead Moroccan Lamb Stew
Cold Roasted Moroccan Spiced Salmon
Israeli Moroccan Couscous
Moroccan Meat Cigars
Moroccan Chicken Thighs
Moroccan Fig Pork Roast
Moroccan-Spiced Pork Roast
Moroccan Black-Eyed Peas (Cowpeas)
Vegetarian Moroccan Stew
Moroccan Harira Soup
Moroccan Lamb Tagine
Moroccan Potato Casserole
Moroccan Mashed Potatoes
Moroccan Chicken Tagine
Moroccan-Spiced Chicken Skewers
Error: None
Error: None
Error: None
Error: None
Error: None
Error: None
Error: None
Error: None
Error: None
Error: None
Error: None
Error: None
Error: None
Error: None
Error: 

TypeError: object of type 'NoneType' has no len()

In [ ]:
data3 = pd.DataFrame(recipes_content2)
data3.head()

In [ ]:
data3.shape

In [ ]:
data3.drop(columns='id', axis=1, inplace=True)
data3.columns

In [ ]:
data3.to_csv('Cleaned Data/Moroccan Data/scrapped_moroccan_recipes3.csv', index=False)

------

In [94]:
# Load your HTML content
with open('recipes datasets/Moroccan Recipes.txt', 'r', encoding='utf-8') as file:
    content = file.read()

soup = BeautifulSoup(content, 'html.parser')

# Extract all the headers (meal names) and their descriptions
meal_data = []
for header in soup.find_all('h3', {'class': 'wp-block-heading headers'}):
    meal_name = header.get_text(strip=True)  # Get meal name
    description = header.find_next('p').get_text(strip=True)  # Get description under the header
    
    meal_data.append({
        'meal_name': meal_name,
        'description': description
    })

In [95]:
data4 = pd.DataFrame(meal_data)
data4.head()

,meal_name,description
0,1. Beef Tagine (Moroccan Beef Stew),Beef Tagine is one of the most comforting and ...
1,2. Zaalouk (Moroccan Eggplant Dip),Are you a fan of delicious dips? Then you shou...
2,3. Harira – Moroccan Lamb and Legume Soup,Another famous Ramadan month meal is this Hari...
3,4. Moroccan Chicken Tagine,Our Moroccan Chicken not only looks super invi...
4,5. Kefta Mkaouara (Moroccan Meatballs),Tagine-made food is always special and has ama...


In [96]:
def clean_text(text):
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'[^a-zA-Z\s,.!?]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

data4['description'] = data4['description'].apply(clean_text)
data4['meal_name'] = data4['meal_name'].apply(clean_text)
data4['meal_name'] = data4['meal_name'].str.strip('. ')

data4.head()

,meal_name,description
0,Beef Tagine,Beef Tagine is one of the most comforting and ...
1,Zaalouk,Are you a fan of delicious dips? Then you shou...
2,Harira Moroccan Lamb and Legume Soup,Another famous Ramadan month meal is this Hari...
3,Moroccan Chicken Tagine,Our Moroccan Chicken not only looks super invi...
4,Kefta Mkaouara,Taginemade food is always special and has amaz...


In [97]:
data4.to_csv('Cleaned Data/Moroccan Data/scrapped_moroccan_recipes4.csv', index=False)

---------------

In [109]:
recipes_content4 = []
id4 = 1

url = "https://www.mymoroccanfood.com/all-recipes"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# all_recipes = soup.find_all('div', {'class': 'recipe-list recipe-archive'})
all_recipes4 = soup.find('div', {'class': 'main-content'})
# print(all_recipes4)
all_recipes4 = soup.find_all('div', {'class': "summary-item"})
print(len(all_recipes4))

for recipe in all_recipes4 :
        body_card = recipe.find('div', {'class':'summary-thumbnail-outer-container'})
        recipe_link = body_card.find('a').attrs['href']
        # print(recipe_link)
        #-------------------------
        response_4 = requests.get(f"https://www.mymoroccanfood.com{recipe_link}")
        soup_4 =  BeautifulSoup(response_4.content, 'html.parser')
        recipe_content = soup_4.find('div', {'id':'content'})
        #-------------------------
        meal_name_tag = recipe_content.find('h1')
        if meal_name_tag:
            meal_name = meal_name_tag.get_text(strip=True)
            print(meal_name)
        #-------------------------
        description_tag = recipe_content.find('div', class_='sqs-html-content')
        if description_tag:
            description = description_tag.get_text(strip=True)
            # print(description)
        #-------------------------
        ingredients = None
        ingredients_section = recipe_content.find(re.compile(r'(h1|h2|h3)'), text=re.compile(r"\s*INGREDIENTS\s*", re.IGNORECASE))
        if ingredients_section:
            ingredients_list = ingredients_section.find_next('ul')
            ingredients = [li.get_text(strip=True) for li in ingredients_list.find_all('li')]
            # print(ingredients)
        #-------------------------
        recipes_content4.append({
            'id': id4, 
            'title': meal_name, 
            'description': description, 
            'ingredients': ingredients, 
        })
        id4 += 1
        
        print('--'*50)


132
Zaalouk Dip
----------------------------------------------------------------------------------------------------


C:\Users\pc\AppData\Local\Temp\ipykernel_19148\1126165718.py:34: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  ingredients_section = recipe_content.find(re.compile(r'(h1|h2|h3)'), text=re.compile(r"\s*INGREDIENTS\s*", re.IGNORECASE))


Peach and tomato salad with orange blossom and honey dressing
----------------------------------------------------------------------------------------------------
Kefta meatballs and tomato sauce tagine with jalapeño & herb yoghurt sauce
----------------------------------------------------------------------------------------------------
Sesame seeds gazelle horns with rainbow sprinkles
----------------------------------------------------------------------------------------------------
Moroccan spiced fish fillets and bell peppers in tomato sauce
----------------------------------------------------------------------------------------------------
Baghrir pancakes with brown butter spiced apples
----------------------------------------------------------------------------------------------------
Ras El Hanout and chocolate chunk pumpkin bread
----------------------------------------------------------------------------------------------------
Sweet Chermoula Whole Roasted Cauliflower
------

AttributeError: 'NoneType' object has no attribute 'find'

In [110]:
data5 = pd.DataFrame(recipes_content4)
data5.head()

,id,title,description,ingredients
0,1,Zaalouk Dip,Morocco’s favourite way to make the most out o...,"[2 large aubergines (600g), 4 medium tomatoes ..."
1,2,Peach and tomato salad with orange blossom and...,If you were wondering what to make for dinner ...,"[3 peaches, segmented, 150g cherry tomatoes (o..."
2,3,Kefta meatballs and tomato sauce tagine with j...,It’s back to school season and I couldn’t be h...,"[2 tablespoons olive oil, 2 garlic cloves, cru..."
3,4,Sesame seeds gazelle horns with rainbow sprinkles,For me gazelle horns mean one thing: Moroccan ...,"[150g sesame seeds, 250g ground almond, 120g c..."
4,5,Moroccan spiced fish fillets and bell peppers ...,Happy new year!!How have you all been doing!? ...,"[2 tablespoons olive oil, 1 large onions, slic..."


In [111]:
data5.shape

(89, 4)

In [112]:
data5.drop(columns='id', axis=1, inplace=True)
data5.columns

Index(['title', 'description', 'ingredients'], dtype='object')

In [113]:
data5.to_csv('Cleaned Data/Moroccan Data/scrapped_moroccan_recipes5.csv', index=False)

---